In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, '../../venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# so that we can import `tools.py`
# Navigate two directories up from the current working directory
module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
# Add this directory to sys.path if it's not already included
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pickle
from tqdm import tqdm

In [3]:
%%time
# loading a sample from the dataset specified in config, in this case 'noanabeshima/TinyStoriesV2'
from tools import get_data_loaders
train_data_loader, val_data_loader = get_data_loaders(
    'noanabeshima/TinyStoriesV2', 
    batch_size=10_000, 
    streaming=True
)

CPU times: user 2.42 s, sys: 284 ms, total: 2.7 s
Wall time: 4.4 s


In [4]:
%%time
batch = next(train_data_loader)
print(len(batch), batch[0])

10000 Once upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They all loved to play and swim together.
One day, Ollie's mom said, "Ollie, hurry and get some fish for dinner!" Ollie swam fast to catch fish. He saw his friend, the duck. "Hi, Ollie!" said the duck. "Hi, duck!" said Ollie. "I need to hurry and catch fish for my family."
While Ollie was catching fish, he found a big shiny stone. He thought, "This is not a fish, but it is so pretty!" Ollie took the shiny stone home to show his family. They all looked at the shiny stone and smiled. The shiny stone made everyone happy, and they forgot about the fish for dinner.
CPU times: user 764 ms, sys: 78.1 ms, total: 842 ms
Wall time: 3.13 s


In [5]:
# turn it into one string instead of a list of strings
combined_string = '\n\n'.join(batch)

# Convert the string to bytes
combined_bytes = combined_string.encode('utf-8')

print(combined_string[:100])
print([b for b in combined_bytes[:100]])
print(combined_bytes.decode('utf-8')[:100])

Once upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They 
[79, 110, 99, 101, 32, 117, 112, 111, 110, 32, 97, 32, 116, 105, 109, 101, 44, 32, 116, 104, 101, 114, 101, 32, 119, 97, 115, 32, 97, 32, 114, 101, 108, 105, 97, 98, 108, 101, 32, 111, 116, 116, 101, 114, 32, 110, 97, 109, 101, 100, 32, 79, 108, 108, 105, 101, 46, 32, 72, 101, 32, 108, 105, 118, 101, 100, 32, 105, 110, 32, 97, 32, 114, 105, 118, 101, 114, 32, 119, 105, 116, 104, 32, 104, 105, 115, 32, 102, 97, 109, 105, 108, 121, 46, 32, 84, 104, 101, 121, 32]
Once upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They 


# Regex
this is a pre-processing stage where we set the rules for what types of characters are allowed to be merged together

In [6]:
import regex as re

In [7]:
# don't ask me the specifics of how this plays out, i just know it's what they used for GPT4
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
# if you want to mess around with building your own tokenizer, then ^this string is one of the things to mess around with

In [8]:
compiled_pattern = re.compile(GPT4_SPLIT_PATTERN, re.UNICODE)
print(compiled_pattern)

regex.Regex("'(?i:[sdmt]|ll|ve|re)|[^\\r\\n\\p{L}\\p{N}]?+\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]++[\\r\\n]*|\\s*[\\r\\n]|\\s+(?!\\S)|\\s+", flags=regex.V0)


In [9]:
# split the text up into text chunks
text_chunks = re.findall(compiled_pattern, combined_string)
print(len(combined_string), len(text_chunks))

8161552 1933930


In [10]:
print(combined_string[:100])
print(text_chunks[:100])

Once upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They 
['Once', ' upon', ' a', ' time', ',', ' there', ' was', ' a', ' reliable', ' otter', ' named', ' Ollie', '.', ' He', ' lived', ' in', ' a', ' river', ' with', ' his', ' family', '.', ' They', ' all', ' loved', ' to', ' play', ' and', ' swim', ' together', '.\n', 'One', ' day', ',', ' Ollie', "'s", ' mom', ' said', ',', ' "', 'Ollie', ',', ' hurry', ' and', ' get', ' some', ' fish', ' for', ' dinner', '!"', ' Ollie', ' swam', ' fast', ' to', ' catch', ' fish', '.', ' He', ' saw', ' his', ' friend', ',', ' the', ' duck', '.', ' "', 'Hi', ',', ' Ollie', '!"', ' said', ' the', ' duck', '.', ' "', 'Hi', ',', ' duck', '!"', ' said', ' Ollie', '.', ' "', 'I', ' need', ' to', ' hurry', ' and', ' catch', ' fish', ' for', ' my', ' family', '."\n', 'While', ' Ollie', ' was', ' catching', ' fish', ',']


In [11]:
# input text preprocessing
ids = [list(ch.encode("utf-8")) for ch in text_chunks] 
ids_backup = ids # saving this for later just to see how much compression we get
print(len(ids), ids[:100])

1933930 [[79, 110, 99, 101], [32, 117, 112, 111, 110], [32, 97], [32, 116, 105, 109, 101], [44], [32, 116, 104, 101, 114, 101], [32, 119, 97, 115], [32, 97], [32, 114, 101, 108, 105, 97, 98, 108, 101], [32, 111, 116, 116, 101, 114], [32, 110, 97, 109, 101, 100], [32, 79, 108, 108, 105, 101], [46], [32, 72, 101], [32, 108, 105, 118, 101, 100], [32, 105, 110], [32, 97], [32, 114, 105, 118, 101, 114], [32, 119, 105, 116, 104], [32, 104, 105, 115], [32, 102, 97, 109, 105, 108, 121], [46], [32, 84, 104, 101, 121], [32, 97, 108, 108], [32, 108, 111, 118, 101, 100], [32, 116, 111], [32, 112, 108, 97, 121], [32, 97, 110, 100], [32, 115, 119, 105, 109], [32, 116, 111, 103, 101, 116, 104, 101, 114], [46, 10], [79, 110, 101], [32, 100, 97, 121], [44], [32, 79, 108, 108, 105, 101], [39, 115], [32, 109, 111, 109], [32, 115, 97, 105, 100], [44], [32, 34], [79, 108, 108, 105, 101], [44], [32, 104, 117, 114, 114, 121], [32, 97, 110, 100], [32, 103, 101, 116], [32, 115, 111, 109, 101], [32, 102, 105, 1

so this regex just splits the text up into all the token ids that are allowed to be merged, meaning that the regex output we saw above is an upper limit on the tokens that we could end up with if we get a large enough vocabulary, rather than a starting point. 

# BPE tokenization

In [12]:
vocab_size = 2048 - 3 # the desired final vocabulary size. -3 for the three special tokens bos, eos, & pad
num_merges = vocab_size - 256

In [13]:
def get_stats(ids, counts=None):
    """
    Given a list of integers, return a dictionary of counts of consecutive pairs
    Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    Optionally allows to update an existing dictionary of counts
    """
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]): # iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    """
    In the list of integers (ids), replace all consecutive occurrences
    of pair with the new integer token idx
    Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
    """
    newids = []
    i = 0
    while i < len(ids):
        # if not at the very last position AND the pair matches, replace it
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [14]:
# now let's actually do it
merges = {} # (int, int) -> int
for i in tqdm(range(num_merges)):
    # count the number of times every consecutive pair appears
    stats = {}
    for chunk_ids in ids:
        # passing in stats will update it in place, adding up counts
        get_stats(chunk_ids, stats)
    # find the pair with the highest count
    pair = max(stats, key=stats.get)
    # mint a new token: assign it the next available id
    idx = 256 + i
    # replace all occurrences of pair in ids with idx
    ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
    # save the merge
    merges[pair] = idx
    #print(f"merge {i+1}/{num_merges}: {pair} -> {idx} had {stats[pair]} occurrences")

100%|███████████████████████████████████████████████████████████████████████| 1789/1789 [1:07:38<00:00,  2.27s/it]


In [15]:
og = sum([len(t) for t in ids_backup])
new = sum([len(t) for t in (ids)])
print("original length:", og)
print("ids length:", new)
print(f"compression ratio: {og / new:.2f}X")

original length: 8161552
ids length: 2176877
compression ratio: 3.75X


In [16]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    # given ids (list of integers), return Python string
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

print(ids[0], decode(ids[0]))

[434] Once


In [17]:
def encode(text):
    # given a string, return list of integers (the tokens)
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break # nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode("Once upon a time,"))

[434, 439, 258, 400, 44]


In [22]:
# Ensure the tokenizers directory exists
if not os.path.exists('./models'):
    os.makedirs('./models')

# Prepare the tokenizer data to be saved
tokenizer_data = {'merges': merges}

# Save the tokenizer data using pickle
with open(f'./models/{vocab_size}.model', 'wb') as f:
    pickle.dump(tokenizer_data, f)

In [21]:
# taking a pre-existing tokenizer and trimming it down to a smaller size
# i basically ran this cell and then one above it multiple times until i got to the smallest possible size (512 - 3 = 509)
vocab_size = ((vocab_size + 3) // 2) - 3 # the -3's account for our special tokens
merges = {k: v for k, v in merges.items() if v < vocab_size}

In [23]:
from tokenizer import load_tokenizer_data, BPE_Tokenizer
vocab_size = 2048
tokenizer_data = load_tokenizer_data(f'models/{vocab_size-3}.model')
tokenizer = BPE_Tokenizer(tokenizer_data['merges'])

In [24]:
prompt = 'Once upon a time there was a boy named Tim.'
print(tokenizer.display(prompt))

tokens = tokenizer.encode(prompt)
print(tokens)

decoded = tokenizer.decode(tokens)
print(decoded)

['Once', ' upon', ' a', ' time', ' there', ' was', ' a', ' boy', ' named', ' Tim', '.']
[2045, 434, 439, 258, 400, 402, 282, 258, 503, 403, 345, 46]
Once upon a time there was a boy named Tim.


In [25]:
for i in range(vocab_size-3):
    print(f"{i}: '{tokenizer.decode([i])}'")

0: ' '
1: ''
2: ''
3: ''
4: ''
5: ''
6: ''
7: ''
8: '
9: '	'
10: '
'
11: ''
12: ''
'3: '
14: ''
15: ''
16: ''
17: ''
18: ''
19: ''
20: ''
21: ''
22: ''
23: ''
24: ''
25: ''
26: ''
27: ''
28: ''
29: ''
30: ''
31: ''
32: ' '
33: '!'
34: '"'
35: '#'
36: '$'
37: '%'
38: '&'
39: '''
40: '('
41: ')'
42: '*'
43: '+'
44: ','
45: '-'
46: '.'
47: '/'
48: '0'
49: '1'
50: '2'
51: '3'
52: '4'
53: '5'
54: '6'
55: '7'
56: '8'
57: '9'
58: ':'
59: ';'
60: '<'
61: '='
62: '>'
63: '?'
64: '@'
65: 'A'
66: 'B'
67: 'C'
68: 'D'
69: 'E'
70: 'F'
71: 'G'
72: 'H'
73: 'I'
74: 'J'
75: 'K'
76: 'L'
77: 'M'
78: 'N'
79: 'O'
80: 'P'
81: 'Q'
82: 'R'
83: 'S'
84: 'T'
85: 'U'
86: 'V'
87: 'W'
88: 'X'
89: 'Y'
90: 'Z'
91: '['
92: '\'
93: ']'
94: '^'
95: '_'
96: '`'
97: 'a'
98: 'b'
99: 'c'
100: 'd'
101: 'e'
102: 'f'
103: 'g'
104: 'h'
105: 'i'
106: 'j'
107: 'k'
108: 'l'
109: 'm'
110: 'n'
111: 'o'
112: 'p'
113: 'q'
114: 'r'
115: 's'
116: 't'
117: 'u'
118: 'v'
119: 'w'
120: 'x'
121: 'y'
122: 'z'
123: '{